<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b657771a9e6815df4830350903cca59093a3226952aae2b3b2bb7d0328f5d0bc
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-07 14:08:19
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 4.75 K (0.03%)
Current PnL: -20.41 L (-13.55%)
CY Booked + Current PnL: -8.97 L (-5.95%)
-------------------
Total profit:  1.44 L
Total loss:  -21.85 L
-------------------
Total Booked + Current PnL: 18.01 L (14.56%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.1%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.92 L (62.27%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 8.02%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115476.0,14972.0,14134.0,0.34,14.90,12.24,28.96,79.0,1.06,0.82,53.92,MH,ATH,METALS
14,BLUESTARCO,2080.00,10.40,50.0,H-MC,2.96,190450.0,25780.0,17559.0,0.62,15.66,9.22,26.31,89.0,1.47,1.35,24.89,X40N,NTT,AC
77,TTKPRESTIG,770.00,93.17,39.0,M-SC,1.35,83257.0,-17520.0,17609.0,0.02,-17.38,21.15,0.09,245.0,-0.99,0.59,9.38,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-22.18,44.0,H-LC,3.91,207719.0,3219.0,20066.0,0.20,1.57,9.66,11.39,10.0,0.16,1.48,10.67,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.29,55.0,H-LC,3.35,217417.0,7051.0,21720.0,1.36,3.35,9.99,13.68,37.0,0.32,1.55,20.43,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,REPCOHOME,880.0,-59.02,59.0,H-SC,3.38,234655.0,-51850.0,309181.0,4.66,-18.10,131.76,89.82,134.0,-0.17,1.67,24.15,XY24,NTT,FINANCE
68,STARHEALTH,761.0,15.38,67.0,H-SC,5.83,257013.0,-24110.0,156984.0,3.27,-8.58,61.08,47.26,144.0,-0.15,1.83,37.70,XY24,NTT,INSURANCE
85,WHIRLPOOL,2270.0,-46.35,35.0,M-SC,1.09,90780.0,-718.0,79469.0,2.29,-0.79,87.54,86.07,223.0,-0.01,0.65,31.83,XR,NTT,DURABLES
6,ASIANTILES,137.0,6555.56,46.0,L-SC,6.54,73048.0,-20762.0,97380.0,2.02,-22.13,133.31,81.67,269.0,-0.21,0.52,41.46,XR,NTT,CERAMICS
21,COFFEEDAY,80.0,-45.67,47.0,L-SC,31.73,80602.0,-32947.0,73001.0,1.82,-29.02,90.57,35.27,268.0,-0.45,0.57,97.27,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HAPPSTMNDS,1488.71,-25.89,17.0,H-SC,11.91,79984.0,-48181.0,158208.0,-4.64,-37.59,197.80,85.85,132.0,-0.30,0.57,0.00,AR,ATH,IT
69,SURYODAY,240.00,60.09,60.0,H-SC,6.61,147349.0,-31722.0,97457.0,-2.52,-17.71,66.14,36.71,135.0,-0.33,1.05,46.20,XR,NTT,BANKS
46,KPIGREEN,731.05,-8.25,29.0,H-SC,6.75,109204.0,-16093.0,75023.0,-2.18,-12.84,68.70,47.03,141.0,-0.21,0.78,36.73,MH,ATH,POWER
70,SYMPHONY,1306.00,-44.56,35.0,M-SC,17.89,118031.0,-53110.0,53055.0,-2.02,-31.03,44.95,-0.03,196.0,-1.00,0.84,0.77,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.00,100.33,35.0,M-SC,2.11,131490.0,-43069.0,43102.0,-1.92,-24.67,32.78,0.02,221.0,-1.00,0.94,14.37,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1231.74,65.0,M-SC,20.88,181185.0,42.0,45423.0,-1.34,0.02,25.07,25.10,235.0,0.00,1.29,45.20,XY24,NTT,HEALTHCARE
85,WHIRLPOOL,2270.0,-46.35,35.0,M-SC,1.09,90780.0,-718.0,79469.0,2.29,-0.79,87.54,86.07,223.0,-0.01,0.65,31.83,XR,NTT,DURABLES
60,SAIL,228.0,43.54,50.0,M-MC,12.34,226805.0,1843.0,165817.0,-0.69,0.82,73.11,74.53,192.0,0.01,1.61,33.34,XY24,BTT,STEEL
35,ICICIPRULI,790.0,-20.77,50.0,H-MC,2.05,136267.0,479.0,42270.0,0.42,0.35,31.02,31.48,107.0,0.01,0.97,12.51,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.70,43.0,M-LC,5.65,150121.0,-824.0,110279.0,0.48,-0.55,73.46,72.51,53.0,-0.01,1.07,5.44,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,65.0,H-SC,13.51,131220.0,-11259.0,30784.0,-0.59,-7.90,23.46,13.70,163.0,-0.37,0.93,54.88,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,39.0,M-SC,1.35,83257.0,-17520.0,17609.0,0.02,-17.38,21.15,0.09,245.0,-0.99,0.59,9.38,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,50.0,H-SC,2.07,222156.0,-47511.0,83842.0,-1.65,-17.62,37.74,13.47,138.0,-0.57,1.58,12.96,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,38.0,H-SC,1.52,140852.0,-35051.0,77074.0,0.02,-19.93,54.72,23.89,149.0,-0.45,1.00,21.64,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.55,42.0,H-MC,6.23,102125.0,-29710.0,71937.0,-0.35,-22.54,70.44,32.03,98.0,-0.41,0.73,14.39,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1231.74,65.0,M-SC,20.88,181185.0,42.0,45423.0,-1.34,0.02,25.07,25.10,235.0,0.00,1.29,45.20,XY24,NTT,HEALTHCARE
60,SAIL,228.00,43.54,50.0,M-MC,12.34,226805.0,1843.0,165817.0,-0.69,0.82,73.11,74.53,192.0,0.01,1.61,33.34,XY24,BTT,STEEL
32,HINDZINC,730.22,30.43,66.0,M-LC,6.21,212606.0,7530.0,105049.0,-0.56,3.67,49.41,54.89,52.0,0.07,1.51,28.65,X5K,ATH,METALS
14,BLUESTARCO,2080.00,10.40,50.0,H-MC,2.96,190450.0,25780.0,17559.0,0.62,15.66,9.22,26.31,89.0,1.47,1.35,24.89,X40N,NTT,AC
84,VOLTAS,1530.00,-2.34,48.0,H-MC,1.42,205005.0,13263.0,24498.0,0.00,6.92,11.95,19.69,99.0,0.54,1.46,14.18,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115476.0,14972.0,14134.0,0.34,14.90,12.24,28.96,79.0,1.06,0.82,53.92,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,71.0,H-MC,10.84,194711.0,14903.0,97511.0,0.44,8.29,50.08,62.52,88.0,0.15,1.38,45.20,XR,NTT,BANKS
32,HINDZINC,730.22,30.43,66.0,M-LC,6.21,212606.0,7530.0,105049.0,-0.56,3.67,49.41,54.89,52.0,0.07,1.51,28.65,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.83,35.0,H-SC,9.35,125875.0,2539.0,129085.0,-0.04,2.06,102.55,106.72,119.0,0.02,0.90,25.92,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-46.35,35.0,M-SC,1.09,90780.0,-718.0,79469.0,2.29,-0.79,87.54,86.07,223.0,-0.01,0.65,31.83,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,30.0,H-LC,9.96,280242.0,-35600.0,148276.0,0.07,-11.27,52.91,35.67,5.0,-0.24,1.99,1.02,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.37,37.0,H-LC,5.78,208472.0,-43296.0,96502.0,0.60,-17.20,46.29,21.13,27.0,-0.45,1.48,12.14,X40,ATH,PAINTS
31,HINDUNILVR,2922.00,-10.34,41.0,H-LC,8.01,251720.0,-5343.0,40477.0,-0.97,-2.08,16.08,13.67,24.0,-0.13,1.79,17.91,XY25,NTT,FMCG
42,ITC,452.00,-39.19,42.0,H-LC,2.15,195991.0,-4147.0,25028.0,0.01,-2.07,12.77,10.44,4.0,-0.17,1.39,3.38,X40,NTT,FMCG
19,CIPLA,1662.74,-22.18,44.0,H-LC,3.91,207719.0,3219.0,20066.0,0.20,1.57,9.66,11.39,10.0,0.16,1.48,10.67,X40N,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,1.03,57.0,H-LC,2.01,162450.0,-23645.0,71121.0,-0.08,-12.71,43.78,25.51,15.0,-0.33,1.16,19.71,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.19,42.0,H-LC,2.15,195991.0,-4147.0,25028.0,0.01,-2.07,12.77,10.44,4.0,-0.17,1.39,3.38,X40,NTT,FMCG
48,LTIM,7230.2,-4.34,54.0,H-LC,2.37,237600.0,-8551.0,87769.0,0.11,-3.47,36.94,32.18,16.0,-0.10,1.69,31.32,X200,ATH,IT
51,NESTLEIND,1377.0,-9.75,54.0,H-LC,3.08,277735.0,12309.0,44493.0,0.44,4.64,16.02,21.40,11.0,0.28,1.98,11.59,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.29,55.0,H-LC,3.35,217417.0,7051.0,21720.0,1.36,3.35,9.99,13.68,37.0,0.32,1.55,20.43,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,30.0,H-LC,9.96,280242.0,-35600.0,148276.0,0.07,-11.27,52.91,35.67,5.0,-0.24,1.99,1.02,X40N,ATH,FMCG
74,TCS,4406.34,-29.65,45.0,H-LC,12.17,280280.0,-65676.0,133918.0,-0.22,-18.98,47.78,19.73,1.0,-0.49,1.99,3.23,X40,ATH,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.15,195991.0,-4147.0,25028.0,0.01,-2.07,12.77,10.44,4.0,-0.17,1.39,3.38,X40,NTT,FMCG
40,INFY,2275.00,-19.96,46.0,H-LC,8.08,312450.0,-438.0,172129.0,-0.62,-0.14,55.09,54.87,3.0,-0.00,2.22,6.50,X40,BTT,IT
30,HCLTECH,1937.36,-1.08,51.0,H-LC,9.84,226158.0,-15766.0,78002.0,1.61,-6.52,34.49,25.73,8.0,-0.20,1.61,6.87,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,1.03,57.0,H-LC,2.01,162450.0,-23645.0,71121.0,-0.08,-12.71,43.78,25.51,15.0,-0.33,1.16,19.71,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,42.0,H-LC,2.15,195991.0,-4147.0,25028.0,0.01,-2.07,12.77,10.44,4.0,-0.17,1.39,3.38,X40,NTT,FMCG
19,CIPLA,1662.74,-22.18,44.0,H-LC,3.91,207719.0,3219.0,20066.0,0.20,1.57,9.66,11.39,10.0,0.16,1.48,10.67,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-17.37,37.0,H-LC,5.78,208472.0,-43296.0,96502.0,0.60,-17.20,46.29,21.13,27.0,-0.45,1.48,12.14,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.29,55.0,H-LC,3.35,217417.0,7051.0,21720.0,1.36,3.35,9.99,13.68,37.0,0.32,1.55,20.43,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.65,45.0,H-LC,12.17,280280.0,-65676.0,133918.0,-0.22,-18.98,47.78,19.73,1.0,-0.49,1.99,3.23,X40,ATH,IT
40,INFY,2275.00,-19.96,46.0,H-LC,8.08,312450.0,-438.0,172129.0,-0.62,-0.14,55.09,54.87,3.0,-0.00,2.22,6.50,X40,BTT,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.15,195991.0,-4147.0,25028.0,0.01,-2.07,12.77,10.44,4.0,-0.17,1.39,3.38,X40,NTT,FMCG
83,VBL,671.64,-22.15,30.0,H-LC,9.96,280242.0,-35600.0,148276.0,0.07,-11.27,52.91,35.67,5.0,-0.24,1.99,1.02,X40N,ATH,FMCG
1,ABB,7934.00,-39.67,52.0,H-LC,7.50,252216.0,-9403.0,128605.0,0.70,-3.59,50.99,45.57,7.0,-0.07,1.79,7.32,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,47.0,L-SC,31.73,80602.0,-32947.0,73001.0,1.82,-29.02,90.57,35.27,268.0,-0.45,0.57,97.27,XR,NTT,HOTELS
49,MASFIN,398.61,-18.25,50.0,H-SC,11.52,93015.0,-4965.0,26565.0,0.62,-5.07,28.56,22.05,152.0,-0.19,0.66,35.20,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115476.0,14972.0,14134.0,0.34,14.90,12.24,28.96,79.0,1.06,0.82,53.92,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,65.0,H-SC,13.51,131220.0,-11259.0,30784.0,-0.59,-7.90,23.46,13.70,163.0,-0.37,0.93,54.88,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.56,146178.0,-3042.0,147216.0,1.55,-2.04,100.71,96.62,208.0,-0.02,1.04,58.20,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.56,146178.0,-3042.0,147216.0,1.55,-2.04,100.71,96.62,208.0,-0.02,1.04,58.20,XY25,NTT,FINANCE
69,SURYODAY,240.00,60.09,60.0,H-SC,6.61,147349.0,-31722.0,97457.0,-2.52,-17.71,66.14,36.71,135.0,-0.33,1.05,46.20,XR,NTT,BANKS
26,GREENPANEL,537.00,233.33,60.0,M-SC,3.69,148913.0,-34165.0,113144.0,0.98,-18.66,75.98,43.14,230.0,-0.30,1.06,40.07,XY24,NTT,MISC
32,HINDZINC,730.22,30.43,66.0,M-LC,6.21,212606.0,7530.0,105049.0,-0.56,3.67,49.41,54.89,52.0,0.07,1.51,28.65,X5K,ATH,METALS
11,BANKINDIA,190.00,-24.22,71.0,H-MC,10.84,194711.0,14903.0,97511.0,0.44,8.29,50.08,62.52,88.0,0.15,1.38,45.20,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.09
1,25,43.34
2,50,75.05


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.06
LC    32.20
MC    23.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.97
X40      14.58
X40N     11.21
XR       10.99
XY25     10.42
AR        8.18
OX40N     7.70
X200      1.69
MH        1.60
X5K       1.51
SR        1.13
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.98
H-LC    25.68
H-MC    20.49
M-SC    13.65
M-LC     5.51
M-MC     2.93
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.75,-6.10,41.07
IT,12.62,-18.56,82.78
FINANCE,10.56,-13.68,68.58
MISC,6.93,-17.92,83.26
BANKS,6.35,-11.58,70.28
PAINTS,5.54,-18.63,36.32
ELECTRICAL,5.38,-8.89,47.09
INSURANCE,3.81,-3.37,42.02
AC,3.50,1.72,14.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3216577.0,22
XR,1364626.0,14
AR,1180114.0,9
X40,789144.0,10
X40N,658350.0,9
OX40N,564495.0,10
XY25,473638.0,7
SR,262657.0,2
X5K,105049.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3717669.0,29
M-SC,1511133.0,17
H-MC,1273432.0,15
H-LC,1181797.0,15
M-LC,375564.0,4
M-MC,370820.0,2
L-SC,259646.0,3
L-MC,59807.0,1
L-LC,41708.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1202007.0      6
M-SC       XY24       850478.0      7
H-SC       AR         811885.0      5
           XR         798620.0      7
H-MC       XY24       568013.0      4
H-LC       X40        505579.0      5
M-MC       XY24       370820.0      2
H-SC       X40N       308488.0      4
           SR         262657.0      2
           OX40N      258989.0      4
H-LC       X40N       217010.0      3
H-MC       X40        210961.0      4
H-LC       AR         199726.0      2
H-MC       XY25       178024.0      2
L-SC       XR         170381.0      2
M-SC       AR         168503.0      2
M-LC       XY24       160236.0      2
M-SC       XY25       147216.0      1
           OX40N      144304.0      4
H-MC       X40N       132852.0      2
M-SC       XR         128028.0      2
M-LC       XR         110279.0      1
H-LC       XY25       106690.0      3
M-LC       X5K        105049.0      1
H-MC       XR          97511.0      1
L-SC       OX40N       89265.0      1
H-LC       X200        87769.0      1
H-SC       MH          75023.0      1
M-SC       X40         72604.0      1
H-MC       OX40N       71937.0      1
H-LC       XY24        65023.0      1
L-MC       XR          59807.0      1
L-LC       XY25        41708.0      1
H-MC       MH          14134.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
